In [ ]:
import sqlglot
import pandas as pd
import numpy as np
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sklearn.cluster import KMeans
from  import MachineLearning
from sqlglot.tokens import Tokenizer

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        # TODO fazer rodar com a minha classe do Kmenas
        
        mlclass = MachineLearning()

        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        # kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        # clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)

class Scatter(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        print("hi")
    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)


# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        KEYWORDS = {
            **Tokenizer.KEYWORDS,
            "VISUALIZE": "VISUALIZE",
            "lineplot": "LINEPLOT",
        }

    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list, "SCATTER": Scatter.from_arg_list}

        # def _parse_visualize(self):
        #     self._match_texts(["VISUALIZE"])  # Reconhece "VISUALIZE"
        #     expression = self._parse_function()  # Captura scatter(cluster), lineplot(x, y)
        #     return self.expression(Scatter, this=expression)
        
        # STATEMENT_PARSERS = {**Parser.STATEMENT_PARSERS, "VISUALIZE": _parse_visualize}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Scatter: lambda self, expr: f"SCATTER({self.sql(expr.this)})",
            # ADD lineplot, decision tree and linear regression
        }

df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT SCATTER(price) FROM resultados_cluster;
"""

expression = sqlglot.parse_one(query, dialect=CustomDialect)

cluster_expr = expression.find(Cluster)

df_clusterizado = cluster_expr.to_cte(df)

print(df_clusterizado)


ModuleNotFoundError: No module named 'ml'